<a href="https://colab.research.google.com/github/Keco28/proyectoModelos/blob/main/proyecto_Modelos2_KevinCorrea_JovanZambrano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

Cargamos la base de datos y mostramos los 10 primeros resultados.

In [5]:
archivo = pd.read_csv('../content/300k.csv', delimiter=',')

FileNotFoundError: ignored

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
archivo[['pokemonId','latitude', 'longitude','appearedDay', 'class']].head(10)

NameError: ignored

In [ ]:
archivo[['pokemonId','latitude', 'longitude','appearedDay', 'class']].describe()

,pokemonId,latitude,longitude,appearedDay,class
count,296021.000000,296021.000000,296021.000000,296021.000000,296021.000000
mean,45.318555,38.774278,-35.704107,4.958675,45.318555
std,39.603926,17.029203,67.398336,1.398349,39.603926
min,1.000000,-42.984504,-159.332033,2.000000,1.000000
25%,16.000000,33.809355,-90.237842,4.000000,16.000000
50%,21.000000,42.246942,-71.119739,5.000000,21.000000
75%,69.000000,49.478332,12.568306,6.000000,69.000000
max,149.000000,68.438473,175.616223,8.000000,149.000000


Lo siguiente que haremos, es dejar las columnas de valores INT para poder facilitar más tarde las operaciones entre matrices.


In [ ]:
archivo_filtrado = archivo[['pokemonId','appearedDay','class']]

# **MÉTODO DE VALIDACIÓN**


In [ ]:
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
def validacion(ytest, ypred, show_r = False):
  mae = mean_absolute_error(ytest, ypred)
  mape = MAPE(ytest, ypred)
  mse = mean_squared_error(ytest, ypred)
  rmse = np.sqrt(mse)
  r2 = r2_score(ypred,ytest)
  if (show_r):
    print("MAE: ", mae)
    print("MAPE: ", mape)yy
    print("Error cuadrático medio: ", mse)
    print("Raiz ECM: ", rmse)
    print("R2: ", r2)
  else:
    return mae, mape, mse, rmse

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(archivo_filtrado, archivo_filtrado, test_size = 0.3, random_state = 0)

In [ ]:
print("Number X_train dataset: ", X_train.shape) 
print("Number y_train dataset: ", y_train.shape) 
print("Number X_test dataset: ", X_test.shape) 
print("Number y_test dataset: ", y_test.shape) 

Number X_train dataset:  (207214, 3)
Number y_train dataset:  (207214, 3)
Number X_test dataset:  (88807, 3)
Number y_test dataset:  (88807, 3)


# **1. METODO DE REGRESIÓN MÚLTIPLE**


In [ ]:
regresion = LinearRegression()
regresion.fit(X_train, y_train)
regresion_predicha= regresion.predict(X_test)
validacion(y_test, regresion_predicha, show_r=True)

MAE:  7.618502488543845e-15
MAPE:  8.290570776314658e-14
Error cuadrático medio:  8.341314916078686e-29
Raiz ECM:  9.133079938377134e-15
R2:  1.0


# **2. MÉTODO DE VENTANA DE PARZEN**


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from scipy.spatial import distance
from sklearn.model_selection import GridSearchCV

In [ ]:
knn = KNeighborsRegressor(n_neighbors = 6, algorithm='ball_tree')
knn.fit(X_train, y_train)

predicted_value = knn.predict(X_test)

print(knn.score(X_train, y_train))
validacion(y_test, predicted_value , show_r= True )

0.9999217574590511
MAE:  0.0006649876198447823
MAPE:  0.012535664021804678
Error cuadrático medio:  0.0003664834562173737
Raiz ECM:  0.019143757630553457
R2:  0.9998617179767479


# **3. Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

In [ ]:
def extract_features(n):
    ext = PCA(n_components=n)
    return ext

In [ ]:
scaler = RobustScaler()
X_scaled = scaler.fit(X_train).transform(X_train)
y_log = np.log(y_train)
pca = extract_features(0.95)
X_scaled=pca.fit_transform(X_scaled)
test_x = pca.transform(X_test)

rf = RandomForestRegressor(n_estimators=1100, max_depth=15)

rf.fit(X_scaled, y_log)
predicted_value = rf.predict(test_x)
validacion(y_test, predicted_value, show_r=True)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but PCA was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


MAE:  27.796708982084084
MAPE:  71.66636076893722
Error cuadrático medio:  2124.254267507054
Raiz ECM:  46.0896329721452
R2:  -1.9409251062671638e+24


# **4. Por vectores de soporte**

In [ ]:
from sklearn.svm import SVR

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [ ]:
est = smf.ols(formula='pokemonId ~ appearedDay', data=archivo_filtrado).fit()
est.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              pokemonId   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     2.029
Date:                Thu, 27 Oct 2022   Prob (F-statistic):              0.154
Time:                        18:29:20   Log-Likelihood:            -1.5091e+06
No. Observations:              296021   AIC:                         3.018e+06
Df Residuals:                  296019   BIC:                         3.018e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      45.6862      0.268    170.350      0.000      45.161      46.212
appearedDay    -0.0741      0.052     -1.424      0.154      -0.176       0.028
==============================================================================
Omnibus:                    40442.834   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            58747.671
Skew:                           1.084   Prob(JB):                         0.00
Kurtosis:                       2.754   Cond. No.                         19.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

en esta parte se realizaron los analisis individuales de los pares de características

In [ ]:
latitude1 = archivo['latitude']
longitude1 = archivo['appearedDay']
est = smf.ols(formula='latitude1 ~ longitude1', data=archivo).fit()
est.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              latitude1   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1444.
Date:                Thu, 27 Oct 2022   Prob (F-statistic):          4.00e-315
Time:                        18:29:20   Log-Likelihood:            -1.2585e+06
No. Observations:              296021   AIC:                         2.517e+06
Df Residuals:                  296019   BIC:                         2.517e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     42.9811      0.115    373.623      0.000      42.756      43.207
longitude1    -0.8484      0.022    -37.995      0.000      -0.892      -0.805
==============================================================================
Omnibus:                   151918.463   Durbin-Watson:                   1.872
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1023757.797
Skew:                          -2.433   Prob(JB):                         0.00
Kurtosis:                      10.702   Cond. No.                         19.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
x = archivo['pokemonId']
y = archivo['latitude']
est = smf.ols(formula=' x~ y', data=archivo).fit()
est.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      x   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     35.53
Date:                Thu, 27 Oct 2022   Prob (F-statistic):           2.51e-09
Time:                        18:29:20   Log-Likelihood:            -1.5091e+06
No. Observations:              296021   AIC:                         3.018e+06
Df Residuals:                  296019   BIC:                         3.018e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     46.3064      0.181    255.823      0.000      45.952      46.661
y             -0.0255      0.004     -5.961      0.000      -0.034      -0.017
==============================================================================
Omnibus:                    40451.058   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            58744.083
Skew:                           1.084   Prob(JB):                         0.00
Kurtosis:                       2.752   Cond. No.                         105.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector


In [ ]:
sfs = SequentialFeatureSelector(regresion, n_features_to_select=2)
sfs.fit(X_test,y_test)


SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=2)

In [ ]:
sfs = SequentialFeatureSelector(knn, n_features_to_select=2)
sfs.fit(X_test,y_test)

SequentialFeatureSelector(estimator=KNeighborsRegressor(algorithm='ball_tree',
                                                        n_neighbors=6),
                          n_features_to_select=2)

In [ ]:
sfs = SequentialFeatureSelector(rf, n_features_to_select=2)
sfs.fit(X_test,y_test)

SequentialFeatureSelector(estimator=RandomForestRegressor(max_depth=15,
                                                          n_estimators=1100),
                          n_features_to_select=2)

In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib import style
style.use('ggplot') or plt.style.use('ggplot')

# Preprocesado y modelado
# ==============================================================================
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Entrenamiento modelo PCA con escalado de los datos
# ==============================================================================
pca_pipe = make_pipeline(StandardScaler(), PCA(), LinearRegression())
pca_pipe.fit(X=X_train, y=y_train)

# Se extrae el modelo entrenado del pipeline
modelo_pca = pca_pipe.named_steps['pca']

In [ ]:
# Predicciones test
# ==============================================================================
predicciones = pca_pipe.predict(X=X_test)


# Error de test del modelo 
# ==============================================================================
rmse_pcr = mean_squared_error(
            y_true  = y_test,
            y_pred  = predicciones,
            squared = False
           )
print("")
print(f"El error (rmse) de test es: {rmse_pcr}")


El error (rmse) de test es: 7.995135010272535e-15
